In [1]:
import sklearn
import numpy as np
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import missingno as msno
import optuna
import gc
from prettytable import PrettyTable
import seaborn as sns
import lightgbm as lgb
from joblib import Parallel, delayed
from tqdm import tqdm
from numexpr import evaluate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import KFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import PowerTransformer, FunctionTransformer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from skopt import BayesSearchCV
from sklearn.impute import KNNImputer
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor, CatBoostClassifier
import warnings
from scipy.stats import pearsonr, chi2_contingency, pointbiserialr
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
os.getcwd()

'C:\\Users\\Jepssen\\0_Mlops\\Untitled Folder'

In [3]:
def load_dataset (filename):
    file_path =os.path.join(os.getcwd(),filename)
    data  = pd.read_csv(file_path)
    return data

In [4]:
train = load_dataset ('train.csv')
test = load_dataset ('test.csv')

In [5]:
print(train.shape, test.shape)

(165034, 14) (110023, 13)


In [6]:
train.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [7]:
# Supposons que 'train' et 'test' sont déjà chargés
#train.drop(columns=["id", "Surname", "CustomerId"], inplace=True)
#test.drop(columns=["id", "Surname"], inplace=True)
y_target = train['Exited']
Xtrain_copy = train.drop(columns=['Exited'])

In [8]:
# Supposons que 'train' et 'test' sont déjà chargés
# Diviser les données
#train.drop(columns=["id", "Surname", "CustomerId"], inplace=True)
#test.drop(columns=["id", "Surname", "CustomerId"], inplace=True)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(train.drop(columns=['Exited']),
                                                train['Exited'],
                                                test_size=0.2,
                                                random_state=0,
                                                stratify=y_target)

In [9]:

# DropColumns Transformer
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns, errors='ignore')

    def get_feature_names_out(self, input_features=None):
        return [col for col in input_features if col not in self.columns]

In [10]:
# FillMissing Transformer
class FillMissing(BaseEstimator, TransformerMixin):
    def __init__(self, target, max_iterations=10):
        self.target = target
        self.max_iterations = max_iterations
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        train = X.copy()
        if self.target in train.columns:
            train = train.drop(columns=self.target)
        
        df = pd.concat([train, y], axis="rows").reset_index(drop=True)
        numeric_features = df.select_dtypes(include=np.number).columns
        categorical_features = df.select_dtypes(include='object').columns

        if len(numeric_features) > 0:
            imputer_numeric = KNNImputer()
            df[numeric_features] = imputer_numeric.fit_transform(df[numeric_features])
        
        if len(categorical_features) > 0:
            for feature in categorical_features:
                mode_value = df[feature].mode()[0]
                df[feature] = df[feature].fillna(mode_value)
        
        return df.iloc[:train.shape[0]].reset_index(drop=True)

    def get_feature_names_out(self, input_features=None):
        return input_features



In [11]:
# HandleRareCategories Transformer
class HandleRareCategories(BaseEstimator, TransformerMixin):
    def __init__(self, col_list):
        self.col_list = col_list
    
    def fit(self, X, y=None):
        self.common_values = {}
        for col in self.col_list:
            self.common_values[col] = set(X[col].value_counts().index)
        return self
    
    def transform(self, X):
        df = X.copy()
        for col in self.col_list:
            common = self.common_values[col]
            df[col] = df[col].apply(lambda x: nearest_val(x, common) if x not in common else x)
        return df

    def get_feature_names_out(self, input_features=None):
        return input_features

def nearest_val(target, common):
    return min(common, key=lambda x: abs(x - target))




In [12]:
# OneHotEncoderCustom Transformer
class OneHotEncoderCustom(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_encode, min_percentage=0.0):
        self.cols_to_encode = cols_to_encode
        self.min_percentage = min_percentage
        self.feature_names_out_ = None
    
    def fit(self, X, y=None):
        self.train_value_counts_ = {}
        self.min_freq_category_ = {}
        self.min_freq_percentage_ = {}

        for col in self.cols_to_encode:
            value_counts = X[col].value_counts(normalize=True)
            self.train_value_counts_[col] = value_counts
            self.min_freq_category_[col] = value_counts.idxmin()
            self.min_freq_percentage_[col] = value_counts.min()
        
        return self
    
    def transform(self, X):
        X_encode = pd.DataFrame(index=X.index)
        for col in self.cols_to_encode:
            ohe_prefix = f"{col}_OHE"
            value_counts = self.train_value_counts_[col]
            min_freq_category = self.min_freq_category_[col]
            min_freq_category_name = f"{ohe_prefix}_{min_freq_category}"

            if self.min_freq_percentage_[col] < self.min_percentage:
                dummies = pd.get_dummies(X[col], prefix=ohe_prefix, prefix_sep='_')
                dummies.drop(columns=min_freq_category_name, errors='ignore', inplace=True)
            else:
                dummies = pd.get_dummies(X[col], prefix=ohe_prefix, prefix_sep='_')

            X_encode = pd.concat([X_encode, dummies], axis=1)
        
        non_encoded_cols = X.drop(columns=self.cols_to_encode)
        X_combined = pd.concat([non_encoded_cols, X_encode], axis=1)
        self.feature_names_out_ = X_combined.columns
        
        return X_combined

    def get_feature_names_out(self, input_features=None):
        encoded_cols = []
        for col in self.cols_to_encode:
            for value in self.train_value_counts_[col].index:
                encoded_cols.append(f"{col}_OHE_{value}")
        return encoded_cols


In [22]:
class ModelTrainer:
    def __init__(self, pipeline, Xtrain, Ytrain, Xtest, Ytest):
        self.pipeline = pipeline
        self.Xtrain = Xtrain
        self.Ytrain = Ytrain
        self.Xtest = Xtest
        self.Ytest = Ytest
        self.label_encoder = LabelEncoder()
        self.best_models = {}
        self.column_names = []
        
    def encode_labels(self):
        """
        Encode les étiquettes de classe en entiers.
        """
        self.Ytrain = self.label_encoder.fit_transform(self.Ytrain.ravel())
        self.Ytest = self.label_encoder.transform(self.Ytest.ravel())
    
    def train_models(self):
        """
        Entraîne les modèles de base sur les données d'entraînement et trouve les meilleurs hyperparamètres.
        """
        # Initialiser les classificateurs
        adaboost_clf = AdaBoostClassifier(random_state=42)
        extratrees_clf = ExtraTreesClassifier(random_state=42)
        gradientboost_clf = GradientBoostingClassifier(random_state=42)
        randomforest_clf = RandomForestClassifier(random_state=42)

        # Définir les grilles de recherche pour les hyperparamètres
        adaboost_params = {'n_estimators': [50, 100]}
        extratrees_params = {'n_estimators': [50, 100]}
        gradientboost_params = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]}
        randomforest_params = {'n_estimators': [50, 100]}
        
        # Initialiser les objets de recherche sur grille
        adaboost_grid = GridSearchCV(adaboost_clf, adaboost_params, cv=3, scoring='accuracy')
        extratrees_grid = GridSearchCV(extratrees_clf, extratrees_params, cv=5, scoring='accuracy')
        gradientboost_grid = GridSearchCV(gradientboost_clf, gradientboost_params, cv=5, scoring='accuracy')
        randomforest_grid = GridSearchCV(randomforest_clf, randomforest_params, cv=5, scoring='accuracy')

        # Effectuer la recherche sur grille pour chaque classificateur
        adaboost_grid.fit(self.Xtrain, self.Ytrain)
        extratrees_grid.fit(self.Xtrain, self.Ytrain)
        gradientboost_grid.fit(self.Xtrain, self.Ytrain)
        randomforest_grid.fit(self.Xtrain, self.Ytrain)

        # Stocker les meilleurs modèles
        self.best_models['adaboost'] = adaboost_grid.best_estimator_
        self.best_models['extratrees'] = extratrees_grid.best_estimator_
        self.best_models['gradientboost'] = gradientboost_grid.best_estimator_
        self.best_models['randomforest'] = randomforest_grid.best_estimator_
        
    def evaluate_models(self):
        """
        Évalue les modèles de base sur l'ensemble de validation.
        """
        for name, model in self.best_models.items():
            test_pred = model.predict(self.Xtest)
            test_accuracy = accuracy_score(self.Ytest, test_pred)
            print(f"{name.capitalize()} Accuracy on validation set: {test_accuracy}")
            
    def train_stacking_model(self):
        """
        Entraîne le modèle de stacking en utilisant les modèles de base comme estimateurs.
        """
        base_classifiers = [(name, model) for name, model in self.best_models.items()]
        self.stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression())
        self.stacking_clf.fit(self.Xtrain, self.Ytrain)
        
    def evaluate_stacking_model(self):
        """
        Évalue le modèle de stacking sur l'ensemble de test.
        """
        stacking_pred = self.stacking_clf.predict(self.Xtest)
        stacking_accuracy = accuracy_score(self.Ytest, stacking_pred)
        print("Stacking Accuracy on test set:", stacking_accuracy)
        
    def run(self):
        """
        Exécute le processus complet d'encodage, d'entraînement, d'évaluation et de test.
        """
        # Transformation des données d'entraînement et de test
        self.Xtrain_enc = self.pipeline.fit_transform(self.Xtrain, self.Ytrain)
        self.Xtest_enc = self.pipeline.transform(self.Xtest)
        
        # Récupérer les noms de colonnes après transformation
        self.column_names = self.pipeline.named_steps['preprocessor'].get_feature_names_out(input_features=self.Xtrain.columns)
        self.column_names = [col.replace('remainder__', '').replace('one_hot_encoding__', '') for col in self.column_names]
        
        # Conversion des données transformées en DataFrame avec les noms de colonnes
        self.Xtrain = pd.DataFrame(self.Xtrain_enc, columns=self.column_names)
        self.Xtest = pd.DataFrame(self.Xtest_enc, columns=self.column_names)
        
        # Encodage des étiquettes
        self.encode_labels()
        
        # Entraînement et évaluation des modèles de base
        self.train_models()
        self.evaluate_models()
        
        # Entraînement et évaluation du modèle de stacking
        self.train_stacking_model()
        self.evaluate_stacking_model()

In [23]:
# Les colonnes à transformer
cont_cols = ["CreditScore", "Age", "Balance", "EstimatedSalary"]
cat_cols_to_encode = ["Geography", "Tenure", "NumOfProducts", "Gender"]
cols_rare = ["Tenure", "NumOfProducts"]
columns_drop =["id", "Surname", "CustomerId"]

# Exemple de pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_columns', DropColumns(columns=["id", "Surname", "CustomerId"]), columns_drop),
        ('fill_missing', FillMissing(target='Exited'), []),
        ('handle_rare_categories', HandleRareCategories(col_list=cols_rare), []),
        ('one_hot_encoding', OneHotEncoderCustom(cols_to_encode=cat_cols_to_encode), cat_cols_to_encode)
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])



In [24]:
# Initialiser et exécuter le processus
model_trainer = ModelTrainer(pipeline, Xtrain, Ytrain, Xtest, Ytest)
model_trainer.run()

Adaboost Accuracy on validation set: 0.8620898597267246
Extratrees Accuracy on validation set: 0.8506983367164541
Gradientboost Accuracy on validation set: 0.8653618929318023
Randomforest Accuracy on validation set: 0.8577574453903717
Stacking Accuracy on test set: 0.8655436725543066


In [ ]:
# Récupérer les noms de colonnes après transformation
column_names = pipeline.named_steps['preprocessor'].get_feature_names_out(input_features=Xtrain.columns)
# Supprimer les préfixes remainder__ et one_hot_encoding__
column_names = [col.replace('remainder__', '').replace('one_hot_encoding__', '') for col in column_names]
# Afficher les noms de colonnes sans les préfixes
print(column_names)

In [ ]:
# Créer une instance de LabelEncoderTransformer
label_encoder = LabelEncoderTransformer()
# Encoder les étiquettes de classe
Ytrain_encoded = label_encoder.fit_transform(Ytrain)
Ytest_encoded = label_encoder.transform(Ytest)
set(Ytest_encoded)

In [ ]:
Xtrain_transformed

In [ ]:
len(column_names)

In [ ]:
Xtrain= pd.DataFrame(Xtrain_transformed, columns=column_names)
Xtrain

In [ ]:
Xtest= pd.DataFrame(Xtest_transformed, columns=column_names)
Xtest

In [ ]:
Xtest.info()

In [ ]:
Xtest.columns

In [ ]:
Xtrain.columns

In [ ]:
# Initialiser les classificateurs
adaboost_clf = AdaBoostClassifier(random_state=42)
extratrees_clf = ExtraTreesClassifier(random_state=42)
gradientboost_clf = GradientBoostingClassifier(random_state=42)
randomforest_clf = RandomForestClassifier(random_state=42)

In [ ]:
# Définir les grilles de recherche pour les hyperparamètres
adaboost_params = {'n_estimators': [50, 100, 200]}
extratrees_params = {'n_estimators': [50, 100, 200]}
gradientboost_params = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]}
randomforest_params = {'n_estimators': [50, 100, 200]}


In [ ]:
# Initialiser les objets de recherche sur grille
adaboost_grid = GridSearchCV(adaboost_clf, adaboost_params, cv=5, scoring='accuracy')
extratrees_grid = GridSearchCV(extratrees_clf, extratrees_params, cv=5, scoring='accuracy')
gradientboost_grid = GridSearchCV(gradientboost_clf, gradientboost_params, cv=5, scoring='accuracy')
randomforest_grid = GridSearchCV(randomforest_clf, randomforest_params, cv=5, scoring='accuracy')


In [ ]:
# Effectuer la recherche sur grille pour chaque classificateur
adaboost_grid.fit(Xtrain, Ytrain)
extratrees_grid.fit(Xtrain, Ytrain)
gradientboost_grid.fit(Xtrain, Ytrain)
randomforest_grid.fit(Xtrain, Ytrain)

In [ ]:
# Sélectionner les meilleurs modèles
best_adaboost_clf = adaboost_grid.best_estimator_
best_extratrees_clf = extratrees_grid.best_estimator_
best_gradientboost_clf = gradientboost_grid.best_estimator_
best_randomforest_clf = randomforest_grid.best_estimator_

In [ ]:
# Faire des prédictions sur l'ensemble de test avec les meilleurs modèles
adaboost_test_pred = best_adaboost_clf.predict(Xtest)
extratrees_test_pred = best_extratrees_clf.predict(Xtest)
gradientboost_test_pred = best_gradientboost_clf.predict(Xtest)
randomforest_test_pred = best_randomforest_clf.predict(Xtest)

In [ ]:
# Évaluer la performance des meilleurs modèles sur l'ensemble de test
adaboost_test_accuracy = accuracy_score(Ytest, adaboost_test_pred)
extratrees_test_accuracy = accuracy_score(Ytest, extratrees_test_pred)
gradientboost_test_accuracy = accuracy_score(Ytest, gradientboost_test_pred)
randomforest_test_accuracy = accuracy_score(Ytest, randomforest_test_pred)


In [ ]:
print("AdaBoost Accuracy on test set:", adaboost_test_accuracy)
print("ExtraTrees Accuracy on test set:", extratrees_test_accuracy)
print("GradientBoost Accuracy on test set:", gradientboost_test_accuracy)
print("RandomForest Accuracy on test set:", randomforest_test_accuracy)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Définir les classificateurs de base pour le stacking
base_classifiers = [
    ('adaboost', best_adaboost_clf),
    ('extratrees', best_extratrees_clf),
    ('gradientboost', best_gradientboost_clf),
    ('randomforest', best_randomforest_clf)
]

In [ ]:
# Initialiser le classificateur de stacking avec une régression logistique comme estimateur final
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression())
# Entraîner le classificateur de stacking sur l'ensemble d'entraînement et de validation combiné
stacking_clf.fit(Xtrain, Ytrain)


In [ ]:
# Faire des prédictions sur l'ensemble de test avec le classificateur de stacking
stacking_test_pred = stacking_clf.predict(Xtest)

In [ ]:
# Évaluer la performance du classificateur de stacking sur l'ensemble de test
stacking_test_accuracy = accuracy_score(Ytest, stacking_test_pred)
print("Stacking Accuracy on test set:", stacking_test_accuracy)

In [ ]:
# Exemple d'utilisation
model_trainer = ModelTrainer(Xtrain, Ytrain, Xtest, Ytest)
model_trainer.run()